In [537]:
# Data Processing
import re
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# NLP
from wordcloud import WordCloud

import spacy
nlp = spacy.load("es_core_news_sm")

# Others
import warnings
warnings.filterwarnings(action='ignore')

## Dataset

In [538]:
df = pd.read_csv('../data/raw/google_maps_all_reviews.csv')

In [539]:
print("Shape:", df.shape)

Shape: (3476, 8)


In [540]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3476 entries, 0 to 3475
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   place_id   3476 non-null   object 
 1   place_url  3476 non-null   object 
 2   review_id  3476 non-null   object 
 3   user_url   3476 non-null   object 
 4   username   0 non-null      float64
 5   stars      3476 non-null   int64  
 6   time       3474 non-null   object 
 7   text       3476 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 217.4+ KB


In [541]:
df.drop(columns=["place_id", "place_url", "user_url", "username", "review_id", "time"], inplace=True)

In [542]:
df

,stars,text
0,5,Costo beneficio correcto. Limpio y comodo de u...
1,5,Buena atención del personal. Muy higiénico. Ri...
2,5,Me encanto el lugar super recomendado La amabi...
3,5,"Un lugar para ejercitarse en el gimnasio, clas..."
4,5,"Un bello lugar para respirar aire puro, nadar,..."
...,...,...
3471,5,Jose Miguel Montes Local Guide · 26 opiniones ...
3472,3,Sandra Carmina Ventura Roldan       Hace...
3473,4,Giovanni Ramirez Garcia Local Guide · 78 opini...
3474,4,Sarita Arana       Hace 8 años


## Filtrado

In [543]:
def normalize_text(text):
    if not isinstance(text, str):
        try:
            import pandas as _pd
            if _pd.isna(text):
                text = ''
            else:
                text = str(text)
        except Exception:
            text = '' if text is None else str(text)

    text = text.lower()                         # Convert to lowercase
    text = re.sub(r'http\S+', '', text)         # Remove URLs
    text = re.sub(r'@\w+', '', text)            # Remove mentions
    text = re.sub(r'#\w+', '', text)            # Remove hashtags
    text = re.sub(r'\s+', ' ', text).strip()    # Remove extra whitespace
    
    convert_dict = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'ü': 'u', 'ñ': 'ni'}
    for accented_char, normal_char in convert_dict.items():
        text = text.replace(accented_char, normal_char)

    return text

In [544]:
df["clean_text"] = df["text"].apply(normalize_text)

In [545]:
check_1 = pd.DataFrame()

check_1 = df[df['clean_text'].str.contains('·')]

print("Removing rows with '·' in text:", check_1.shape[0])
df = df.drop(check_1.index)
print("New shape:", df.shape)

Removing rows with '·' in text: 732
New shape: (2744, 3)


In [546]:
check_2 = pd.DataFrame()

re_expression = [r'hace \d+ anios', r'hace un anio', r'hace un mes', r'hace \d+ meses']

for expr in re_expression:
    temp = df[df['clean_text'].str.contains(expr)]
    check_2 = pd.concat([check_2, temp])
    print(f"Number of rows with '{expr}':", temp.shape[0])

print("\nRemoving", check_2.shape[0], "rows from df")
df = df.drop(index=check_2.index)
print("New shape:", df.shape)

Number of rows with 'hace \d+ anios': 515
Number of rows with 'hace un anio': 32
Number of rows with 'hace un mes': 2
Number of rows with 'hace \d+ meses': 13

Removing 562 rows from df
New shape: (2182, 3)


In [547]:
df

,stars,text,clean_text
0,5,Costo beneficio correcto. Limpio y comodo de u...,costo beneficio correcto. limpio y comodo de u...
1,5,Buena atención del personal. Muy higiénico. Ri...,buena atencion del personal. muy higienico. ri...
2,5,Me encanto el lugar super recomendado La amabi...,me encanto el lugar super recomendado la amabi...
3,5,"Un lugar para ejercitarse en el gimnasio, clas...","un lugar para ejercitarse en el gimnasio, clas..."
4,5,"Un bello lugar para respirar aire puro, nadar,...","un bello lugar para respirar aire puro, nadar,..."
...,...,...,...
3299,5,Excelente,excelente
3303,5,bantab,bantab
3305,5,Desde,desde
3306,5,Cicatrización,cicatrizacion


## Pre Procesamiento

In [548]:
def split_into_sentences(text):
    sentences = re.split(r'[.!?]+', text)
    
    cleaned_sentences = []
    for sentence in sentences:
        sentence = sentence.strip()
        sentence = re.sub(r'[^a-zA-Z0-9áéíóúüñÁÉÍÓÚÜÑ\s]', '', sentence)
        if sentence and sentence != '':
            cleaned_sentences.append(sentence)
    
    return cleaned_sentences

In [549]:
df['sentences'] = df['clean_text'].apply(split_into_sentences)

In [550]:
df

,stars,text,clean_text,sentences
0,5,Costo beneficio correcto. Limpio y comodo de u...,costo beneficio correcto. limpio y comodo de u...,"[costo beneficio correcto, limpio y comodo de ..."
1,5,Buena atención del personal. Muy higiénico. Ri...,buena atencion del personal. muy higienico. ri...,"[buena atencion del personal, muy higienico, r..."
2,5,Me encanto el lugar super recomendado La amabi...,me encanto el lugar super recomendado la amabi...,[me encanto el lugar super recomendado la amab...
3,5,"Un lugar para ejercitarse en el gimnasio, clas...","un lugar para ejercitarse en el gimnasio, clas...",[un lugar para ejercitarse en el gimnasio clas...
4,5,"Un bello lugar para respirar aire puro, nadar,...","un bello lugar para respirar aire puro, nadar,...",[un bello lugar para respirar aire puro nadar ...
...,...,...,...,...
3299,5,Excelente,excelente,[excelente]
3303,5,bantab,bantab,[bantab]
3305,5,Desde,desde,[desde]
3306,5,Cicatrización,cicatrizacion,[cicatrizacion]


In [551]:
def lemmatize_spacy(sentences):
    if isinstance(sentences, str):
        sentences = [sentences]
    if not isinstance(sentences, (list, tuple)):
        return []

    out = []
    for s in sentences:
        doc = nlp(s)
        lem = " ".join([token.lemma_ for token in doc if token.is_alpha])
        out.append(lem)
    return out

In [552]:
df['lemm_sentences'] = df['sentences'].apply(lemmatize_spacy)

In [553]:
df

,stars,text,clean_text,sentences,lemm_sentences
0,5,Costo beneficio correcto. Limpio y comodo de u...,costo beneficio correcto. limpio y comodo de u...,"[costo beneficio correcto, limpio y comodo de ...","[costo beneficio correcto, limpio y comodo de ..."
1,5,Buena atención del personal. Muy higiénico. Ri...,buena atencion del personal. muy higienico. ri...,"[buena atencion del personal, muy higienico, r...","[buen atencion del personal, mucho higienico, ..."
2,5,Me encanto el lugar super recomendado La amabi...,me encanto el lugar super recomendado la amabi...,[me encanto el lugar super recomendado la amab...,[yo encantar el lugar super recomendado el ama...
3,5,"Un lugar para ejercitarse en el gimnasio, clas...","un lugar para ejercitarse en el gimnasio, clas...",[un lugar para ejercitarse en el gimnasio clas...,[uno lugar para ejercitar él en el gimnasio cl...
4,5,"Un bello lugar para respirar aire puro, nadar,...","un bello lugar para respirar aire puro, nadar,...",[un bello lugar para respirar aire puro nadar ...,[uno bello lugar para respirar aire puro nadar...
...,...,...,...,...,...
3299,5,Excelente,excelente,[excelente],[excelente]
3303,5,bantab,bantab,[bantab],[bantab]
3305,5,Desde,desde,[desde],[desde]
3306,5,Cicatrización,cicatrizacion,[cicatrizacion],[cicatrizacion]


In [554]:
df.to_csv('../data/processed/reviews.csv', index=False)